### Kirchhoff solver for electrical circuits

The Kirchhoff Laws are highly used for solving electric circuits, that means, to determine the intensities that pass throught each branch of the circuit, given the resistances and pottencial differences between all nodes. 

Ideas:

- Graph direct modelling 

- Nodes, branches and meshes

- Linear algebra to solve the system.

-----

In [1586]:
import networkx as nx
import sympy as sym
import numpy as np


class ElectricalCircuit():
    """ Electrical circuit modelling using an undirect weighted MultiGraph. 
    
    Features:
    ----------
    - Captures and analyzes an electrical circuit given by it's branches, 
    in aim to get the current value which flows for each branch. 
    
    - Applies Kirchhoff Laws over the nodes and meshes of the circuit, 
    and resolves the linear resulting system, if possible.
    
    - Uses an internal graph representation of the circuit, shaping the 
    nodes, branches and meshes of the circuit as the nodes, edges and 
    cycles of the graph.
    
    Edge weights will contain the branches information, only supporting 
    (for the moment):
    
    a) the resistance value 'R': 
            
        The total resistance value inside the branch. It's zero by default, 
        when there are no components with an appreciable resistance value.
            
    b) the voltage value 'V': 
        
        The total pottential difference given by the voltage sources inside 
        the branch. Because the sources have two positions (direct or inverse), 
        we consider a positive value when the source is direct in respect the 
        ascending order '<' between edge connected nodes, and negative if not. 
            
    c) the intensity value 'I': 
        
        The intensity value of the current that pass throught the branch. 
        Represents one unknown of the Kirchhoff linear system, and its value 
        is updated after every change. If the system isn't stable, just 
        outputs a '???' value. 
              
    Attributes:
    ------------
    nodes    : list of the circuit nodes.
    
    branches : dictionary of the circuit branches. Contains the above 
               'R', 'V' and 'I' values.
    
    meshes   : list of the circuit meshes or loops, as the base cicles 
               of the internal graph.
    
    With these three attributes (and the subjacent connection), one can solve 
    the circuit by just extracting manually the necessary equations.
    
    Instead, this class only needs the branches dictionary passed as an initial 
    parameter, and it will calculate the intensities automatically. Of course, 
    it still needs the work of translating correctly all the circuit components.
    
    """       
    
    # Constants
    V_units = 'V'
    R_units = 'kO'
    I_units = 'mA'
    
    
    # Constructor
    def __init__(self, edgelist = [], precision = 4):
        """ Builds the circuit by passing a list of branches with their attributes. 
            Empty circuit by default. """
        
        self._circuit = nx.MultiGraph()
        self._precision = precision
        self._type = 'numeric'
        self._meshes = []
        
        # Add all branches
        for edge in edgelist:
            self.add_branch((edge[0], edge[1]), **(edge[2].copy()))
    
    
    # Branch adder
    def add_branch(self, edge, **edgedict):
        """ Adds a new branch to the circuit. Also sets its 'R' and 'V' values. """
        # Insertion preconditions + preprocessing
        self.__order_between_nodes(edge)
        self.__supported_dict_keys(edgedict)
        self.__supported_value_types(edgedict)
        
        # Insertion
        self._circuit.add_edge(edge[0], edge[1], **edgedict)
            
        # Updates
        self.__mesh_update()
        self.__kirchhoff_update()
        
        
    def __order_between_nodes(self, edge): 
        """ Edge has to respect node's ascending order. 
        Raises an error if edge[0] ≥ edge[1]. """
        if edge[0] >= edge[1]: raise AssertionError(
        "Branch does not satisfy the order '<' between nodes.")
            
            
    def __supported_dict_keys(self, edgedict): 
        """ Edge dictionary must have two of the supported variables: 
        'V', 'R', 'I', to determine the unknown variable. """
        if ('V' in edgedict and 'R' in edgedict and 'I' not in edgedict): 
            edgedict['unknown'] = 'I' # Intensity as the unknown variable
            
        elif ('V' not in edgedict and 'R' in edgedict and 'I' in edgedict): 
            edgedict['unknown'] = 'V' # Voltage as the unknown variable
            
        elif ('V' in edgedict and 'R' not in edgedict and 'I' in edgedict): 
            edgedict['unknown'] = 'R' # Resistance as the unknown variable
            
        else: raise AttributeError("Branch dictionary must have one and only one" +
                    " undefined key variable (the unknown).")
            
            
    def __supported_value_types(self, edgedict): 
        """ Edge dictionary value types must be either numeric or string 
        (or both in tuple format). """
        
        for key in ['V', 'R', 'I']:
            if key in edgedict:
                # numeric type (default)
                if type(edgedict[key]) == int or type(edgedict[key]) == float:
                    edgedict[key] = sym.Float(edgedict[key], self._precision) 
                
                # symbolic type
                elif type(edgedict[key]) == str:
                    edgedict[key] = sym.Symbol(edgedict[key]) 
                    self._type = 'simbolic'
                    
                elif ((type(edgedict[key]) == tuple and len(edgedict[key]) == 2) 
                      and (type(edgedict[key][0]) == int or type(edgedict[key][0]) == float) 
                        and type(edgedict[key][1]) == str): 
                    edgedict[key] = edgedict[key][0] * sym.Symbol(edgedict[key][1])
                    self._type = 'simbolic'
                    
                else: raise TypeError(str(key) + " value type for branch dictionary " 
                        + "must be numeric (int, float), string or both in tuple.")
            
            
    # Branch deleter
    def del_branch(self, edge):
        """ Removes the branch specified from the circuit. Has to give a key. """
        self._circuit.remove_edge(edge[0], edge[1], key = edge[2])
        
        # Updates
        self.__mesh_update()
        self.__kirchhoff_update()
        
        
    # Getter for nodes
    @property
    def nodes(self):
        """ Getter for the nodes list of the circuit. """
        return list(self._circuit.nodes)
    
    
    # Getter for branches
    @property
    def branches(self):
        """ Getter for the edges dictionary of the circuit. """
        return dict(self._circuit.edges)
    
    
    # Getter for meshes
    @property
    def meshes(self):
        """ Getter for the base cycles list of the circuit. """
        return self._meshes
    
    
    # Voltage difference
    def potential_diff(self, nodeA, nodeB):
        """ Returns the pottential difference between nodeA and nodeB, as the 
        pottential of nodeA with respect nodeB: V_AB = (V_A - V_B). """
        
        diff = 0
        
        # For all edges in a biconnected path from B to A,
        nodes = nx.shortest_path(self._circuit, nodeB, nodeA)
        for branch in [(nodes[i], nodes[i+1], 0) for i in range(len(nodes) - 1)]:

            # intensity direction
            if branch in self.branches:
                diff += self._circuit.edges[branch]['V']
                diff -= self._circuit.edges[branch]['I'] * self._circuit.edges[branch]['R']

            # intensity inverse direction
            if (branch[1], branch[0], branch[2]) in self.branches:
                diff -= self._circuit.edges[branch]['V']
                diff += self._circuit.edges[branch]['I'] * self._circuit.edges[branch]['R']
                    
        return diff
        
        
    # Thévenin transformation
    def thevenin(self, edge):
        """ Thévenin... """
        
        # Remember branch dictionary for safety delete
        edge_attr = dict(self._circuit.edges[edge])
        self.del_branch(edge)
        
        # Gets the Thévenin equivalent voltage as the potential difference
        # between that branch connected nodes.
        V = self.potential_diff(edge[1], edge[0])
        
        
        # Builds a list of branches with their respective resistances
        resistances = [(branch[0], branch[1], self._circuit.edges[branch]['R']) 
                           for branch in self._circuit.edges]
        
        # While all resistances haven't been merged.
        while len(resistances) > 1:
            print(resistances)
            
            j = 0
            # Merge all parallel connected resistances
            for i in range(len(resistances) - 1):
                R = 1 / resistances[i][2]
                while (i+1 < len(resistances) and ((resistances[i][0], resistances[i][1]) == 
                        (resistances[i+1][0], resistances[i+1][1]))):
                    R += 1 / resistances.pop(i+1)[2]
                    
                resistances[i] = (resistances[i][0], resistances[i][1], 1/R)
                if i+1 == len(resistances): break
            
            print(resistances)
            # Merge one pair of serie connected resistances
            for i in range(len(resistances) - 1):
                for j in range(i+1, len(resistances)):
                    if resistances[i][1] == resistances[j][0]: # And there's no one more!!
                        new_branch = (resistances[i][0], resistances[j][1], 
                                          resistances[i][2] + resistances[j][2])
                        for k in range(i, j):
                            if ((resistances[k][0], resistances[k][1]) <= (new_branch[0], new_branch[1]) and 
                                (new_branch[0], new_branch[1]) <= (resistances[k+1][0], resistances[k+1][1])):
                                resistances.insert(k+1, new_branch)
                                break
                                
                        # Remove the parents and breaks twice
                        resistances.pop(i)
                        resistances.pop(j)
                        break
                else: 
                    continue
                break
                    
        # Gets the Thévenin equivalent resistance as the result
        # of combining all resistances.
        R = resistances.pop()
        
        # Finally, push the branch back to the circuit.
        del edge_attr[edge_attr['unknown']]
        del edge_attr['unknown']
        self.add_branch(edge, **edge_attr)
        
        return V, R
    
        
    # Str representation
    def __str__(self):
        """ Shows the circuit as a pictographic drawing, by calling the 
        circuit_view or branches_view depending on its size and type. """
        
        # Full view representation
        if (self._type == 'numeric' and self._precision <= 4 and 
            len(self._circuit.nodes) <= 4):
            return self.circuit_view()
        
        # Short view representation
        else: return self.branches_view()
        
       
    # Circuit view (not sure if well labeled)
    def circuit_view(self):
        """ Returns the full circuit view, printing nodes as column 
        separators with homogeneous branches. 
        Thinked for short numeric type circuits (less than 4 nodes). """
        
        node_idx = {node:i for i, node in enumerate(self.nodes)} # Index remembering
        sep = "\t\t\t\t\t" # Separator between nodes
        
        # Str representation. Default branch as default lines, for branch separations 
        circuit_pattern = sep.join([f"{node}" for node in self.nodes]) + "\n"
        
        # For each branch between nodes
        for j, branch in enumerate(self._circuit.edges):
            # get the branch nodes index.
            min_idx = min(node_idx[branch[0]], node_idx[branch[1]])
            max_idx = max(node_idx[branch[0]], node_idx[branch[1]])
            
            # Join and append branch components
            branch_components = []
            for component, value in self._circuit.edges[branch].items():
                # If V is not the unknown
                if component == 'V' and value:
                    branch_components.append("(" + str(value) + " " + self.V_units + ")")
                if component == 'R' and value:
                    branch_components.append("[" + str(value) + " " + self.R_units + "]")
                if component == 'I' and value:
                    branch_components.append("\\" + str(value) + " " + self.I_units + "\\")
            branch_components = "---".join(branch_components)
            branch_components = branch_components.center(40 * (max_idx - min_idx) - 1, "-")
            
            # Add two default separator lines
            branch_pattern = [""] + [sep for node in self.nodes][:-1] + [""]
            circuit_pattern += "|".join(branch_pattern) + "\n"
            circuit_pattern += "|".join(branch_pattern) + "\n"
            
            # Add the branch line
            circuit_pattern += "|".join(branch_pattern[:min_idx + 1])
            circuit_pattern += "+" + branch_components + "+"
            circuit_pattern += "|".join(branch_pattern[max_idx + 1:]) + "\n"
        
        return circuit_pattern
    
    
    # Branches view
    def branches_view(self):
        """ Short circuit view, as a row list of branches. Suitable for 
        simbolic type circuits, or dense circuits (more than 4 nodes). """
        
        circuit_pattern = "" # Str representation of the branches
        
        # For each branch between nodes
        for j, branch in enumerate(self._circuit.edges):
            
            # Join and append branch components
            branch_components = []
            for component, value in self._circuit.edges[branch].items():
                # If V is not the unknown
                if component == 'V' and value:
                    branch_components.append("(" + str(value) + " " + self.V_units + ")")
                if component == 'R' and value:
                    branch_components.append("[" + str(value) + " " + self.R_units + "]")
                if component == 'I' and value:
                    branch_components.append("\\" + str(value) + " " + self.I_units + "\\")
            branch_components = "--------".join(branch_components)
            circuit_pattern += str(branch) + ": (" + str(branch[0]) + ")-------" 
            circuit_pattern += branch_components + "-------(" + str(branch[1]) + ")\n\n"
        return circuit_pattern
        
        
    # Cycle base calculator
    def __mesh_update(self):
        """ Mesh update, every time that the circuit has changed. 
        Gets all the graph base cycles, combined with the multiple edge cycles.
        """
        
        # Get all base cycles for the undirect multiple graph as:
        self._base_circuit = nx.Graph(self._circuit)
        self._meshes = []

        # the sum of all the base cycles for the undirect simple graph
        for cycle in nx.cycle_basis(self._base_circuit):
            self._meshes += [[(cycle[i-1], cycle[i], 0) for i in range(len(cycle))]]

        # with all the inner cycles for the undirect multiple graph
        for edge in self._base_circuit.edges:
            n = len(self._circuit.adj[edge[0]][edge[1]])
            self._meshes += [[(edge[0], edge[1], i), (edge[1], edge[0], i + 1)] 
                             for i in range(n - 1)]
        
        
    # Kirchhoff solver
    def __kirchhoff_update(self):
        """ Applies Kirchhoff Laws for soving the circuit, by updating 
        the unknowed values for each branch. """
        
        # Number of nodes and edges.
        N = len(self._circuit.nodes)
        E = len(self._circuit.edges)
        
        if not E: return # For empty circuits!!
        
        # Sympy matrixes for the lineal resulting system AX = B.
        A = sym.Matrix.zeros(E)
        B = sym.Matrix.zeros(E, 1)
        X = sym.symbols(f'x:{E}')
        
        # Apply Kirchhoff laws to get the linear system
        for i in range(E):
            for j, branch in enumerate(self._circuit.edges):
                
                # 1st Kirchhoff Law (charge conservation): 
                # the sum of currents meeting at a node is zero.
                if i < (N - 1):
                    if (branch[1] == list(self._circuit.nodes)[i]): # intensity in
                        
                        # Be sure of treating the correct unknown ('I', 'V' or 'R')
                        if (self._circuit.edges[branch]['unknown'] == 'I'): A[i, j] = 1
                        else: B[i] -= self._circuit.edges[branch]['I']
                            
                    if (branch[0] == list(self._circuit.nodes)[i]): # intensity out
                        
                        # Be sure of treating the correct unknown ('I', 'V' or 'R')
                        if (self._circuit.edges[branch]['unknown'] == 'I'): A[i, j] = -1
                        else: B[i] += self._circuit.edges[branch]['I']
                     
                # 2nd Kirchhoff Law (energy conservation): 
                # the directed sum of the voltages around any mesh is zero.
                else:
                    # direct branch
                    if branch in self.meshes[i - (N - 1)]:
                        
                        # Be sure of treating the correct unknown ('I', 'V' or 'R')
                        if (self._circuit.edges[branch]['unknown'] == 'I'):
                            A[i, j] = - self._circuit.edges[branch]['R']
                            B[i] -= self._circuit.edges[branch]['V']
                        
                        if (self._circuit.edges[branch]['unknown'] == 'V'): 
                            A[i, j] = 1
                            B[i] += self._circuit.edges[branch]['R'] * self._circuit.edges[branch]['I']
                        
                        if (self._circuit.edges[branch]['unknown'] == 'R'):
                            A[i, j] = - self._circuit.edges[branch]['I']
                            B[i] -= self._circuit.edges[branch]['V']
                        
                    # inverse branch
                    if (branch[1], branch[0], branch[2]) in self.meshes[i - (N - 1)]: 
                        
                        # Be sure of treating the correct unknown ('I', 'V' or 'R')
                        if (self._circuit.edges[branch]['unknown'] == 'I'):
                            A[i, j] = self._circuit.edges[branch]['R']
                            B[i] += self._circuit.edges[branch]['V']
                            
                        if (self._circuit.edges[branch]['unknown'] == 'V'):
                            A[i, j] = -1
                            B[i] -= self._circuit.edges[branch]['R'] * self._circuit.edges[branch]['I']
                            
                        if (self._circuit.edges[branch]['unknown'] == 'R'):
                            A[i, j] = self._circuit.edges[branch]['I']
                            B[i] += self._circuit.edges[branch]['V']
        
        # Solves the linear system for symbolic coefficients 
        # or numeric values and gets the solution
        solution_space = sym.linsolve((A, B), X)
        self.solved = bool(solution_space)
        for solution in solution_space: break
        
        # Adds the solutions back to branches
        for j, branch in enumerate(self._circuit.edges):
            variable = self._circuit.edges[branch]['unknown']
            if self.solved:  self._circuit.edges[branch][variable] = solution[j]
            else:  self._circuit.edges[branch][variable] = sym.S.NaN

#### Tests:

In [1569]:
# Empty circuit test
empty_circuit = ElectricalCircuit()
print('Nodes:', empty_circuit.nodes)
print('Branches:', empty_circuit.branches)
print('Meshes:', empty_circuit.meshes)
print(empty_circuit)

Nodes: []
Branches: {}
Meshes: []




In [1506]:
# Base circuit test
base_circuit = ElectricalCircuit([(1, 2, {'R': 'R1', 'V': 3}), (1, 2, {'R':0, 'V':'V2'}), (2, 3, {'R': 34, 'V': -3})])
print('Nodes:', base_circuit.nodes)
print('Branches:', base_circuit.branches)
print('Meshes:', base_circuit.meshes)
print(base_circuit)

Nodes: [1, 2, 3]
Branches: {(1, 2, 0): {'R': R1, 'V': 3.000, 'unknown': 'I', 'I': (-V2 + 3.0)/R1}, (1, 2, 1): {'R': 0.0, 'V': V2, 'unknown': 'I', 'I': (V2 - 3.0)/R1}, (2, 3, 0): {'R': 34.00, 'V': -3.000, 'unknown': 'I', 'I': 0}}
Meshes: [[(1, 2, 0), (2, 1, 1)]]
(1, 2, 0): (1)-------[R1 kO]--------(3.000 V)--------\(-V2 + 3.0)/R1 mA\-------(2)

(1, 2, 1): (1)-------(V2 V)--------\(V2 - 3.0)/R1 mA\-------(2)

(2, 3, 0): (2)-------[34.00 kO]--------(-3.000 V)-------(3)




In [1507]:
# __order_between_nodes test
try:
    base_circuit.add_branch((2, 1), R = 20, V = 30)
except AssertionError as e:
    print('Test error message: ' + str(e))

Test error message: Branch does not satisfy the order '<' between nodes.


In [1508]:
# __supported_dict_keys test
try:
    base_circuit.add_branch((1, 2), V = 30)
except AttributeError as e:
    print('Test error message: ' + str(e))

Test error message: Branch dictionary must have one and only one undefined key variable (the unknown).


In [1509]:
# __supported_value_types test
try:
    base_circuit.add_branch((1, 2), V = (), I = 38)
except TypeError as e:
    print('Test error message: ' + str(e))

Test error message: V value type for branch dictionary must be numeric (int, float), string or both in tuple.


In [1510]:
# add_branch, del_branch test

empty_circuit.add_branch((1, 2), R = 0.0, V = 0.0)
print(empty_circuit)

try:
    empty_circuit.del_branch((1, 2))
except IndexError as e:
    print('Test error message: ' + str(e))
    
empty_circuit.del_branch((1, 2, 0))
print(empty_circuit)

1					2
|					|
|					|
+---------------------------------------+

Test error message: tuple index out of range
1					2



In [1511]:
# Numeric correctness test

# Intensity unknown
numeric = ElectricalCircuit(precision = 5)
numeric.add_branch((1, 2), R = 0.0, V = -3)
numeric.add_branch((1, 2), R = 3, V = 10)
numeric.add_branch((1, 3), R = 10, V = -20)
numeric.add_branch((2, 3), R = 0.0, V = 100)
numeric.add_branch((2, 3), R = 200, V = 200)
print(numeric)

# Voltage unknown
numeric2 = ElectricalCircuit()
numeric2.add_branch((1, 2), R = 0.0, I = -3)
numeric2.add_branch((1, 2), R = 3, I = 10)
numeric2.add_branch((1, 3), R = 10, V = -20)
numeric2.add_branch((2, 3), R = 0.0, V = 100)
numeric2.add_branch((2, 3), R = 200, V = 200)
print(numeric2)

# Resistance unknown
numeric3 = ElectricalCircuit()
numeric3.add_branch((1, 2), V = 0.0, I = -3)
numeric3.add_branch((1, 2), V = 3, I = 10)
numeric3.add_branch((1, 3), V = 10, R = -20)
numeric3.add_branch((2, 3), V = 0.0, R = 100)
numeric3.add_branch((2, 3), V = 200, R = 200)
print(numeric3)

(1, 2, 0): (1)-------(-3.0000 V)--------\7.3667 mA\-------(2)

(1, 2, 1): (1)-------[3.0000 kO]--------(10.000 V)--------\4.3333 mA\-------(2)

(1, 3, 0): (1)-------[10.000 kO]--------(-20.000 V)--------\-11.700 mA\-------(3)

(2, 3, 0): (2)-------(100.00 V)--------\11.200 mA\-------(3)

(2, 3, 1): (2)-------[200.00 kO]--------(200.00 V)--------\0.50000 mA\-------(3)


1					2					3
|					|					|
|					|					|
+--------\-3.000 mA\---(-50.00 V)-------+					|
|					|					|
|					|					|
+--[3.000 kO]---\10.00 mA\---(-20.00 V)-+					|
|					|					|
|					|					|
+---------------------[10.00 kO]---(-20.00 V)---\-7.000 mA\---------------------+
|					|					|
|					|					|
|					+---------(100.0 V)---\6.500 mA\--------+
|					|					|
|					|					|
|					+--[200.0 kO]---(200.0 V)---\0.5000 mA\-+

1					2					3
|					|					|
|					|					|
+--------\-3.000 mA\---[90.00 kO]-------+					|
|					|					|
|					|					|
+--(3.000 V)---\10.00 mA\---[-26.70 kO]-+					|
|					|					|
|					|					|
+----

In [1512]:
# Symbolic support test

#  Intensity unknown
symbolic = ElectricalCircuit()
symbolic.add_branch(('A', 'B'), R = (2, 'R'), V = 'V2')
symbolic.add_branch(('A', 'C'), R = (2, 'R'), V = 0)
symbolic.add_branch(('A', 'D'), R = (2, 'R'), V = 'V4')
symbolic.add_branch(('B', 'C'), R = 'R', V = (-1, 'V3'))
symbolic.add_branch(('B', 'C'), R = (2, 'R'), V = 'V1')
symbolic.add_branch(('B', 'D'), R = (2, 'R'), V = 0)
symbolic.add_branch(('B', 'D'), R = (2, 'R'), V = 0)
print(symbolic)

# Voltage unknown
symbolic2 = ElectricalCircuit()
symbolic2.add_branch((1, 2), R = 'R', V = 'V1')
symbolic2.add_branch((1, 2), R = 'R', I = 'I2')
symbolic2.add_branch((1, 2), R = 'R', V = 0)
print(symbolic2)

# Resistance unknown
symbolic3 = ElectricalCircuit()
symbolic3.add_branch((1, 2), V = 'V1', I = 'I1')
symbolic3.add_branch((1, 2), V = (-1, 'V2'), R = 'R2')
symbolic3.add_branch((1, 2), I = 'I3', R = 'R3')
print(symbolic3)

('A', 'B', 0): (A)-------[2*R kO]--------(V2 V)--------\(3*V1 + 17*V2 - 6*V3 - 8*V4)/(58*R) mA\-------(B)

('A', 'C', 0): (A)-------[2*R kO]--------\(-5*V1 - 9*V2 + 10*V3 - 6*V4)/(58*R) mA\-------(C)

('A', 'D', 0): (A)-------[2*R kO]--------(V4 V)--------\(V1 - 4*V2 - 2*V3 + 7*V4)/(29*R) mA\-------(D)

('B', 'C', 0): (B)-------[R kO]--------(-V3 V)--------\(-8*V1 + 3*V2 - 13*V3 + 2*V4)/(29*R) mA\-------(C)

('B', 'C', 1): (B)-------[2*R kO]--------(V1 V)--------\(21*V1 + 3*V2 + 16*V3 + 2*V4)/(58*R) mA\-------(C)

('B', 'D', 0): (B)-------[2*R kO]--------\(-V1 + 4*V2 + 2*V3 - 7*V4)/(58*R) mA\-------(D)

('B', 'D', 1): (B)-------[2*R kO]--------\(-V1 + 4*V2 + 2*V3 - 7*V4)/(58*R) mA\-------(D)


(1, 2, 0): (1)-------[R kO]--------(V1 V)--------\(-I2*R + V1)/(2*R) mA\-------(2)

(1, 2, 1): (1)-------[R kO]--------\I2 mA\--------(3*I2*R/2 + V1/2 V)-------(2)

(1, 2, 2): (1)-------[R kO]--------\-(I2*R + V1)/(2*R) mA\-------(2)


(1, 2, 0): (1)-------(V1 V)--------\I1 mA\--------[(-R2*(I1 +

In [1513]:
# Potential difference test

# Numeric support
print('Potential diff V12 for numeric2 circuit:', numeric3.potential_diff(1, 2))
      
# Symbolic support
print('Potential diff VAB for symbolic circuit:', symbolic.potential_diff('A', 'B'))

Potential diff V12 for numeric2 circuit: -270.0
Potential diff VAB for symbolic circuit: 3*V1/29 - 12*V2/29 - 6*V3/29 - 8*V4/29


In [1514]:
# views test:

print("Empty circuit:\n")
print(empty_circuit.branches_view())
print(empty_circuit.circuit_view())

print("\nBase circuit:\n")
print(base_circuit.branches_view())
print(base_circuit.circuit_view())

print("\nNumeric circuit:\n")
print(numeric.branches_view())
print(numeric.circuit_view())

print("\nSymbolic circuit:\n")
print(symbolic.branches_view())
print(symbolic.circuit_view())

Empty circuit:


1					2


Base circuit:

(1, 2, 0): (1)-------[R1 kO]--------(3.000 V)--------\(-V2 + 3.0)/R1 mA\-------(2)

(1, 2, 1): (1)-------(V2 V)--------\(V2 - 3.0)/R1 mA\-------(2)

(2, 3, 0): (2)-------[34.00 kO]--------(-3.000 V)-------(3)


1					2					3
|					|					|
|					|					|
+[R1 kO]---(3.000 V)---\(-V2 + 3.0)/R1 mA\+					|
|					|					|
|					|					|
+------(V2 V)---\(V2 - 3.0)/R1 mA\------+					|
|					|					|
|					|					|
|					+--------[34.00 kO]---(-3.000 V)--------+


Numeric circuit:

(1, 2, 0): (1)-------(-3.0000 V)--------\7.3667 mA\-------(2)

(1, 2, 1): (1)-------[3.0000 kO]--------(10.000 V)--------\4.3333 mA\-------(2)

(1, 3, 0): (1)-------[10.000 kO]--------(-20.000 V)--------\-11.700 mA\-------(3)

(2, 3, 0): (2)-------(100.00 V)--------\11.200 mA\-------(3)

(2, 3, 1): (2)-------[200.00 kO]--------(200.00 V)--------\0.50000 mA\-------(3)


1					2					3
|					|					|
|					|					|
+-------(-3.0000 V)---\7.3667 mA\-------+					|
|					|					|
|					|

In [1588]:
# thevenin test

thevenin = ElectricalCircuit()
thevenin.add_branch(('A', 'B'), R = (2, 'R'), V = 'V2')
thevenin.add_branch(('A', 'C'), R = (2, 'R'), V = 0)
thevenin.add_branch(('A', 'D'), R = (2, 'R'), V = 'V4')
thevenin.add_branch(('B', 'C'), R = 'R', V = (-1, 'V3'))
thevenin.add_branch(('B', 'C'), R = (2, 'R'), V = 'V1')
thevenin.add_branch(('B', 'D'), R = (2, 'R'), V = 0)
thevenin.add_branch(('B', 'D'), R = (2, 'R'), V = 0)
thevenin.add_branch(('A', 'B'), R = (7, 'R'), V = '(V6-V5)')
print(thevenin)
thevenin.thevenin(('A', 'B', 1))

('A', 'B', 0): (A)-------[2*R kO]--------(V2 V)--------\(-24*(V6-V5) + 21*V1 + 143*V2 - 42*V3 - 56*V4)/(454*R) mA\-------(B)

('A', 'B', 1): (A)-------[7*R kO]--------((V6-V5) V)--------\(29*(V6-V5) + 3*V1 - 12*V2 - 6*V3 - 8*V4)/(227*R) mA\-------(B)

('A', 'C', 0): (A)-------[2*R kO]--------\(-18*(V6-V5) - 41*V1 - 63*V2 + 82*V3 - 42*V4)/(454*R) mA\-------(C)

('A', 'D', 0): (A)-------[2*R kO]--------(V4 V)--------\(-8*(V6-V5) + 7*V1 - 28*V2 - 14*V3 + 57*V4)/(227*R) mA\-------(D)

('B', 'C', 0): (B)-------[R kO]--------(-V3 V)--------\(6*(V6-V5) - 62*V1 + 21*V2 - 103*V3 + 14*V4)/(227*R) mA\-------(C)

('B', 'C', 1): (B)-------[2*R kO]--------(V1 V)--------\(6*(V6-V5) + 165*V1 + 21*V2 + 124*V3 + 14*V4)/(454*R) mA\-------(C)

('B', 'D', 0): (B)-------[2*R kO]--------\(8*(V6-V5) - 7*V1 + 28*V2 + 14*V3 - 57*V4)/(454*R) mA\-------(D)

('B', 'D', 1): (B)-------[2*R kO]--------\(8*(V6-V5) - 7*V1 + 28*V2 + 14*V3 - 57*V4)/(454*R) mA\-------(D)


[('A', 'B', 2*R), ('A', 'C', 2*R), ('A', 'D', 2*R

KeyboardInterrupt: 

-------

In [1450]:
prueba = ElectricalCircuit()
prueba.add_branch((1, 2), R = 0, V = 0)
prueba.add_branch((1, 2), R = 0, V = 0)
prueba.add_branch((1, 3), R = 0, V = 0)
prueba.add_branch((1, 3), R = 0, V = 0)
prueba.add_branch((1, 4), R = 0, V = 0)
prueba.add_branch((1, 4), R = 0, V = 0)
prueba.add_branch((2, 3), R = 0, V = 0)
prueba.add_branch((2, 3), R = 0, V = 0)
prueba.add_branch((2, 4), R = 0, V = 0)
prueba.add_branch((3, 4), R = 0, V = 0)
print(prueba)

1					2					3					4
|					|					|					|
|					|					|					|
+--------\-x1 + x6 + x7 + x8 mA\--------+					|					|
|					|					|					|
|					|					|					|
+----------------\x1 mA\----------------+					|					|
|					|					|					|
|					|					|					|
+----------------------------\-x3 - x6 - x7 + x9 mA\----------------------------+					|
|					|					|					|
|					|					|					|
+------------------------------------\x3 mA\------------------------------------+					|
|					|					|					|
|					|					|					|
+---------------------------------------------------\-x5 - x8 - x9 mA\--------------------------------------------------+
|					|					|					|
|					|					|					|
+--------------------------------------------------------\x5 mA\--------------------------------------------------------+
|					|					|					|
|					|					|					|
|					+----------------\x6 mA\----------------+					|
|					|					|					|
|					|					|					|
|					+----------------\x7 mA\----------------+					|
|					|					|					|
|			

In [1457]:
A = prueba.nodes
print(A)

[1, 2, 3, 4]


In [1458]:
A.pop()

4

In [1459]:
print(A)

[1, 2, 3]


In [1460]:
print(prueba.nodes)

[1, 2, 3, 4]


In [1540]:
prueba = nx.MultiGraph()
prueba.add_edge(1, 2, R = 0, V = 0)
prueba.add_edge(1, 2, R = 0, V = 0)
prueba.add_edge(1, 3, R = 0, V = 0)
prueba.add_edge(1, 3, R = 0, V = 0)
prueba.add_edge(1, 4, R = 0, V = 0)
prueba.add_edge(1, 4, R = 0, V = 0)
prueba.add_edge(2, 3, R = 0, V = 0)
prueba.add_edge(2, 3, R = 0, V = 0)
prueba.add_edge(2, 4, R = 0, V = 0)
prueba.add_edge(3, 4, R = 0, V = 0)
prueba.add_edge(1, 2, R = 0, V = 0)

2

In [1470]:
prueba.adj[2]

AdjacencyView({1: {0: {'R': 0, 'V': 0}, 1: {'R': 0, 'V': 0}}, 3: {0: {'R': 0, 'V': 0}, 1: {'R': 0, 'V': 0}}, 4: {0: {'R': 0, 'V': 0}}})

In [1541]:
prueba.edges

MultiEdgeView([(1, 2, 0), (1, 2, 1), (1, 2, 2), (1, 3, 0), (1, 3, 1), (1, 4, 0), (1, 4, 1), (2, 3, 0), (2, 3, 1), (2, 4, 0), (3, 4, 0)])

In [1476]:
for i in range(-1):
    print(i)

In [1480]:
zero, one = (0, 1)
one

1

In [1522]:
len({'A': 1, 'B': 2})

2

In [1538]:
A = prueba.adj[1][2]
print(A)
A[1][2][0]['visited'] = True
print(A)
print(prueba.adj)

{0: {'R': 0, 'V': 0}, 1: {'R': 0, 'V': 0}}


KeyError: 2

In [1566]:
s = 'holaadeu'
for i in range(len(s)):
    for j in range(i+1, len(s)):
        if s[i] == s[j]:
            answer = (i, j)
            print('Founded!')
            break
        print(i, j)
    else:
        continue
    break

0 1
0 2
0 3
0 4
0 5
0 6
0 7
1 2
1 3
1 4
1 5
1 6
1 7
2 3
2 4
2 5
2 6
2 7
Founded!


In [1573]:
A = [1, 2, 3, 4]
A.pop(2)
A.insert(2, 9)
A

[1, 2, 9, 4]